In [1]:
import torch
from torch import nn

import numpy as np
from numpy import dtype

# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU is available


In [2]:
## Generate data


def XOR_data(n_datapoints=100, seq_len_range=(1, 4)):
    inputs = np.zeros([n_datapoints, seq_len_range[1], 1], dtype=np.float32)
    outputs = np.zeros([n_datapoints, 1], dtype=np.float32)
    for i in range(n_datapoints):
        # Generate input sequences
        seq_len = np.random.randint(seq_len_range[0], seq_len_range[1] + 1)
        for j in range(seq_len_range[1]):
            if j < seq_len:
                bit = np.random.choice([0, 1])
                inputs[i, j, 0] = bit
            else:
                inputs[i, j, 0] = np.nan

        # Compute output
        xor = np.nansum(inputs[i]) % 2
        outputs[i] = xor

    return inputs, outputs


inputs, outputs = XOR_data(n_datapoints=100, seq_len_range=(1, 4))
inputs = torch.from_numpy(inputs).to(device)
outputs = torch.from_numpy(outputs).to(device)

In [3]:
class Model(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(Model, self).__init__()

        # Defining some parameters
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        # Defining the layers
        self.rnn = nn.RNN(
            input_size, hidden_dim, n_layers, batch_first=True, nonlinearity="relu"
        )
        self.fc = nn.Linear(hidden_dim, output_size)

    def forward(self, x):

        batch_size = x.size(0)
        max_seq_len = x.size(1)

        # Initializing hidden state for first input using method defined below
        hidden = self.init_hidden(batch_size)

        # Passing in the input and hidden state into the model and obtaining outputs
        out, hidden = self.rnn(x, hidden)
        # Reshaping the outputs such that it can be fit into the fully connected layer
        # out = out.contiguous().view(-1, self.hidden_dim)

        ## Select last output
        indices = max_seq_len - 1 - torch.unsqueeze(out.isnan().sum(axis=1), dim=1)
        out = torch.gather(out, 1, indices)
        out = torch.squeeze(out, dim=2)

        out = self.fc(out)

        return out, hidden

    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim, device=device)
        return hidden

In [4]:
# Instantiate the model with hyperparameters
model = Model(input_size=1, output_size=1, hidden_dim=5, n_layers=1)
# We'll also set the model to the device that we defined earlier (default is CPU)
model.to(device)

# Define hyperparameters
n_epochs = 1000
lr = 0.01

# Define Loss, Optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

# Training Run
for epoch in range(1, n_epochs + 1):
    print(f"Epoch: {epoch}")
    # for param in model.parameters():
    #     print(param.data)
    optimizer.zero_grad()  # Clears existing gradients from previous epoch
    output, hidden = model(inputs)
    loss = criterion(torch.squeeze(output), torch.squeeze(outputs))
    loss.backward()  # Does backpropagation and calculates gradients
    optimizer.step()  # Updates the weights accordingly

    if epoch % 10 == 0:
        print("Epoch: {}/{}.............".format(epoch, n_epochs), end=" ")
        print("Loss: {:.4f}".format(loss.item()))

Epoch: 1
Epoch: 2
Epoch: 3
Epoch: 4
Epoch: 5
Epoch: 6
Epoch: 7
Epoch: 8
Epoch: 9
Epoch: 10
Epoch: 10/1000............. 

/opt/conda/conda-bld/pytorch_1659484808560/work/aten/src/ATen/native/cuda/ScatterGatherKernel.cu:144: operator(): block: [0,0,0], thread: [96,0,0] Assertion `idx_dim >= 0 && idx_dim < index_size && "index out of bounds"` failed.
/opt/conda/conda-bld/pytorch_1659484808560/work/aten/src/ATen/native/cuda/ScatterGatherKernel.cu:144: operator(): block: [0,0,0], thread: [97,0,0] Assertion `idx_dim >= 0 && idx_dim < index_size && "index out of bounds"` failed.
/opt/conda/conda-bld/pytorch_1659484808560/work/aten/src/ATen/native/cuda/ScatterGatherKernel.cu:144: operator(): block: [0,0,0], thread: [98,0,0] Assertion `idx_dim >= 0 && idx_dim < index_size && "index out of bounds"` failed.
/opt/conda/conda-bld/pytorch_1659484808560/work/aten/src/ATen/native/cuda/ScatterGatherKernel.cu:144: operator(): block: [0,0,0], thread: [99,0,0] Assertion `idx_dim >= 0 && idx_dim < index_size && "index out of bounds"` failed.
/opt/conda/conda-bld/pytorch_1659484808560/work/aten/src/ATen/native/cuda/ScatterGat

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [ ]:
def predict(model, sequence):
    input = torch.unsqueeze(
        torch.from_numpy(np.array([sequence], dtype=np.float32)), dim=2
    ).to(device)
    out, hidden = model(input)
    return out

In [ ]:
## Predict on dataset
index = 1
input = inputs[index]
output = torch.squeeze(outputs[index])
prediction, _ = model(torch.unsqueeze(input, dim=0))
prediction = torch.squeeze(prediction)
print(f"Input:{input}")
print(f"Output:{output}")
print(f"Prediction:{prediction}")
print(criterion(prediction, output))

Input:tensor([[0.],
        [0.],
        [1.],
        [0.]], device='cuda:0')
Output:1.0
Prediction:1.0000176429748535
tensor(3.1127e-10, device='cuda:0', grad_fn=<MseLossBackward0>)


In [ ]:
print(predict(model, [1, 1, 1, 1, 0, 0, 1]))

tensor([[[-0.2220]]], device='cuda:0', grad_fn=<ViewBackward0>)


In [ ]:
## REMEMBER np.nan was set to 0, so this is effectivly just a 4 sequence length dataset